In [1]:
import os
import os

api_key = os.getenv("OPENAI_API_KEY")

In [4]:
! pip install numexpr

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [17]:
import os
from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

chain = LLMMathChain.from_llm(llm=llm)

print(chain.run("2+6等于几？"))

Answer: 8


In [19]:
template = "请用中文解释这个词语：{word}"
prompt = PromptTemplate(input_variables=["word"], template=template)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({"word": "男人"}))

C:\Users\25746\AppData\Local\Temp\ipykernel_11232\3594235110.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


男人是指生理上具有男性特征的人类个体，通常具有较强的体力和阳刚气质，也具有独立、坚强、责任感等特质。在社会角色中，男人通常被认为是家庭的经济支柱和保护者，承担着养家糊口、保护家庭成员等责任。男人在社会中扮演着重要的角色，是家庭、社会和国家的支柱之一。


In [21]:
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import (
    CallbackManagerForChainRun
)
from langchain.chains.base import  Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import  BaseLanguageModel


In [23]:
class wiki_article_chain(Chain):
    prompt:BasePromptTemplate
    llm:BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) -> List[str]:
        return self.prompt.input_variables
    
    @property
    def output_keys(self) -> List[str]:
        return [self.out_key]
    
    def _call(
        self,
        inputs:Dict[str,Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    ) -> Dict[str,Any]:
        prompt_value = self.prompt.format_prompt(**inputs)
        #print("prompt_value:",prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        #print("response:",response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self) -> str:
        return "wiki_article_chain"
       

In [37]:
from langchain.chat_models import  ChatOpenAI
from langchain.prompts import  PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(
        temperature=0
    ),
)
result = chain.run({"topic":"降本增效"})
print(result)

降本增效（英文：Cost reduction and efficiency improvement）是指企业或组织通过优化资源配置和管理，降低成本，提高生产效率和经营效益的一种管理方法。降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。

降本增效的目标是通过降低生产成本和提高生产效率，实现企业的盈利最大化。在实施降本增效的过程中，企业需要进行全面的成本分析，找出造成成本增加的原因，并采取相应的措施进行改进。这些措施可以包括优化生产流程、提高员工效率、降低物流成本、节约能源等。

降本增效的方法有很多种，企业可以根据自身的情况选择适合的方法进行实施。其中，技术创新是提高生产效率的重要手段之一。通过引入先进的生产技术和设备，企业可以提高生产效率，降低生产成本。此外，企业还可以通过优化供应链管理、精简组织结构、提高员工培训等方式来实现降本增效。

降本增效不仅可以帮助企业提高盈利能力，还可以提高企业的竞争力和市场地位。通过降本增效，企业可以提高产品质量和服务水平，满足客户需求，赢得客户信赖。同时，降本增效还可以提高企业的可持续发展能力，为企业未来的发展奠定坚实基础。

总的来说，降本增效是企业管理中非常重要的一环，可以帮助企业提高竞争力，实现可持续发展。企业应该不断探索创新，积极实施降本增效措施，提高生产效率，降低成本，实现经营效益的最大化。
